In [1]:
from openstack import connection, identity
import copy
import re

In [2]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn

In [3]:
class OSmap(object):
    
    def __init__ (self, connection='', OS=True, with_flavors=True, with_volumes=True, hypervisor_details=False):
        'Инит функция'
        if OS == True:
            self.__conn = connection
            self.__sl = [i for i in self.__conn.compute.servers(all_tenants=True)]
            self.__hl = [i for i in self.__conn.compute.hypervisors()]
            
            if with_flavors == True:
                self.__fd = self.__get_fl_dict()
                self.dict_flavors = copy.deepcopy(self.__fd)
            
            if with_volumes == True:
                self.__vd = self.__get_vl_dict()
                self.dict_volumes = copy.deepcopy(self.__vd)
                
                
            self.dict_hypervisors = self.__get_hp_dict(details=hypervisor_details)
            self.dict_servers = self.__get_vm_dict(flavors=with_flavors)
            self.list_names  = self.__get_names()
    
    
    def get_vm_by_name(self, vmn=''):
        'Функция возвращает объект ВМ на основании имени'
        for i in self.__sl:
            if i.name == vmn:
                return i 
    
    def get_vm_by_id(self, vmn=''):
        'Функция возвращает объект ВМ на основании ID'
        for i in self.__sl:
            if i.id == vmn:
                return i 
    
        
    
    def __get_hp_dict(self, details=False):
        'Функция возвращает словарь гипервизоров'
        d = {}
        for i in self.__hl:
            if details == True:
                d[i.name]=self.__conn.compute.get_hypervisor(i.id).to_dict()
            else:
                d[i.name]=i.to_dict()
        return d   
           
    def __get_fl_dict(self):
        'Функция возвращает словарь флейворов'
        d = {}
        for i in set([i.flavor['id'] for i in self.__sl]):
            d[i]=self.__conn.compute.get_flavor(flavor=i).to_dict()
        return d
    
    def __get_vl_dict(self):
        'Функция возвращает словарь вольюмов'
        d = {}
        for i in [i for i in self.__conn.block_store.volumes(all_tenants=True)]:
            d[i.id]=i.to_dict()
        return d
    
    
    def __get_names (self):
        'Функция возвращает список имен ВМ'
        a = [i.name for i in self.__sl]
        return a
    
            
    def __get_vm_dict(self,flavors=True):
        'Возвращает словарь с описанием ВМ из OS + cpu & ram'
        d = {}
        for i in self.__sl:
            d[i.name] = self.get_vm_by_name(vmn=i.name).to_dict()
            if flavors == True:
                d[i.name]['cpu'] = self.__fd[i.flavor['id']]['vcpus']
                d[i.name]['ram'] = self.__fd[i.flavor['id']]['ram']
        return d
        

In [4]:
conn = connection_to_cluster('admin')

In [5]:
#%%time
#x = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False)

In [6]:
#%%time
#y = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False)

In [7]:
#%%time
#z = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True)

In [8]:
#%%time
#x1 = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False, hypervisor_details=True)

In [9]:
#%%time
#y1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False, hypervisor_details=True)

In [10]:
%%time
z1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True, hypervisor_details=True)

CPU times: user 5.52 s, sys: 239 ms, total: 5.76 s
Wall time: 40 s


In [11]:
#lGroupPrefixTypes

In [12]:
#lGroupPrefix

In [13]:
#lRealVmName

In [14]:
#dGroups = {}
#for prefix in lGroupPrefix:
#   sTemp = re.escape(prefix) + r"-\d{2,}"
#    #print(prefix)
#    regexp = re.compile(sTemp, re.IGNORECASE)
#    dGroups[prefix]=[]
#    for vm in lRealVmName:
#        if regexp.match(vm):
#            #print(vm)
#            dGroups[prefix].append(vm)

In [15]:
#dGroups

In [16]:
#lGroupsNull = [i for i in dGroups.keys() if dGroups[i] == []]
#lGroupsNull

In [17]:
#sd[z1.get_vm_by_id(vid).name]
#vd['2d2e33a4-5c0f-48e4-9b1e-1cb10e264803']

In [18]:
#lGroupedVm = [i for j in dGroups.keys() for i in dGroups[j]]

In [19]:
#len(set(lGroupedVm)), len(lRealVmName)

In [20]:
#difflists = (set(lRealVmName) - set(lGroupedVm))
#for i in sorted(difflists):
#    print(i)

In [21]:
#vd['68e3161d-42c9-4440-be83-25d9652d403a']

In [22]:
#regexp_string = r"\d{1,}$"
#regexp = re.compile(regexp_string)
#difflists1 = [re.sub(regexp,'',i) for i in difflists]

In [23]:
#len(difflists1), len(set(difflists1))

In [24]:
#for i in set(difflists1):
#    print(i)

## Копируем данные из класса. Создаем словарь групп

In [25]:
## Типы распределений
## 1 - распределение по зонам доступности
## 2 - распределение по ВУ
## 3 - распределение по модулю 2 (четные на одной ЗД, нечетные на другой ЗД)
## 4 - распределение по модулю 3 (1,2,3 на разных ЗД)

In [26]:
dRealVm = copy.deepcopy(z1.dict_servers)

In [27]:
lRealVmName = copy.deepcopy(z1.list_names)

In [28]:
dRealHyper = copy.deepcopy(z1.dict_hypervisors)

In [29]:

with open('./group_prefix', 'r') as f:
    lGroupPrefix = f.readlines()
    lGroupPrefix = [i.replace('\n','') for i in lGroupPrefix]
f.close()


with open('./group_prefixes_via_types', 'r') as f:
    lGroupPrefixTypes = f.readlines()
    lGroupPrefixTypes = [i.replace('\n','') for i in lGroupPrefixTypes]
    lGroupPrefixTypes = [i.split(',') for i in lGroupPrefixTypes]
f.close()


with open('./z1', 'r') as f:
    lAZ1 = f.readlines()
    lAZ1 = [i.replace('\n','') for i in lAZ1]
f.close()


with open('./z2', 'r') as f:
    lAZ2 = f.readlines()
    lAZ2 = [i.replace('\n','') for i in lAZ2]
f.close()
    
with open('./z3', 'r') as f:
    lAZ3 = f.readlines()
    lAZ3 = [i.replace('\n','') for i in lAZ3]
f.close()

with open('./very_new_cn_all', 'r') as f:
    lCN = f.readlines()
    lCN = [i.replace('\n','') for i in lCN]
    lCN = [i.split(',') for i in lCN]
f.close()

with open('./very_new_cn_migr', 'r') as f:
    lMigr = f.readlines()
    lMigr = [i.replace('\n','') for i in lMigr]
    lMigr = [i.split(',') for i in lMigr]
    lMigr = [i[1] for i in lMigr]
f.close()

In [30]:

dGroups = {}
for lprefix in lGroupPrefixTypes:
    prefix_name=lprefix[0]
    prefix_type=int(lprefix[1])
    sTemp = re.escape(prefix_name) + r"-\d{2,}"
    #print(prefix)
    regexp = re.compile(sTemp, re.IGNORECASE)
    dGroups[prefix_name]={}
    dGroups[prefix_name]['type'] = prefix_type
    dGroups[prefix_name]['vms'] = []
    for vm in lRealVmName:
        if regexp.match(vm):
            #print(vm)
            dGroups[prefix_name]['vms'].append(vm)
    dGroups[prefix_name]['vm_count'] = len(dGroups[prefix_name]['vms'])

In [31]:
#[[i, dGroups[i]['vm_count']] for i in dGroups.keys() if dGroups[i]['type'] == 1]


In [217]:
### основные функции по созданию словарей

def create_dict_computes_clean(z1,z2,z3,cn,migr):
    rd={}
    for i in cn:
        hname = i[0]
        hlabel = i[1]
        #print(hlabel,hname)
        if hlabel in z1:
            hzone=int('1')
        elif hlabel in z2:
            hzone=int('2')
        elif hlabel in z3:
            hzone=int('3')
       
        if hlabel in migr:
            hmigr=True
        else: 
            hmigr=False
        
        rd[hname]={}
        rd[hname]['migr']=hmigr
        rd[hname]['label']=hlabel
        rd[hname]['az']=hzone
        rd[hname]['total_cpu']=dRealHyper[hname]['vcpus']
        rd[hname]['total_ram']=dRealHyper[hname]['memory_size']
        rd[hname]['free_cpu']=dRealHyper[hname]['vcpus']
        rd[hname]['free_ram']=dRealHyper[hname]['memory_size']
        rd[hname]['vms']=[]
        rd[hname]['vm_count']=len(rd[hname]['vms'])
        
    return rd
        
def create_dict_vm_with_groups(vdict,gdict):
    rd={}
    for v in vdict.keys():
        rd[v]={}
        rd[v]['cpu']=vdict[v]['cpu']
        rd[v]['ram']=vdict[v]['ram']
        rd[v]['cn']=vdict[v]['hypervisor_hostname']
        rd[v]['cn_new'] = ''
        rd[v]['group']=''
        for g in gdict.keys():
            if v in gdict[g]['vms']:
                rd[v]['group'] = g
                continue
            
    return rd

def create_dict_groups_via_az(gdict):
    rd={}
    rd=copy.deepcopy(gdict)
    for g in gdict:
        rd[g]['az']={}
        rd[g]['az'][1] = int('0')
        rd[g]['az'][2] = int('0')
        rd[g]['az'][3] = int('0')
        rd[g]['az_even'] = ''
        rd[g]['az_odd'] = ''

    
    return rd

In [261]:
##
## Функции распределения и поиска пула доступных гипервизоров
##



###### Необходимо сделать - описать дистрибуцию уровня 2 и 1, дистрибуцию для остальных машин (тип 0). 


def distributuion_init(gdict,vdict,hdict,vms_on_migr):
    global dUseGroups, dUseVms, dUseHyper
    dUseGroups = copy.deepcopy(gdict)
    dUseVms    = copy.deepcopy(vdict)
    dUseHyper  = copy.deepcopy(hdict)
    dl=[]
    for v in dVirtualVm.keys():
        vmname = v
        if dVirtualVm[v]['group'] != '':
            vmtype = dVirtualGroups[dVirtualVm[v]['group']]['type']
        else:
            vmtype= int('0')
        dl.append([vmname,vmtype])
    
    distribution(vl=sorted(dl, key = lambda x: float(x[1]), reverse=True))
    
    
def distribution(vl):
    for v in vl:
        vmname = v[0]
        vmtype = v[1]
        if vmname in vms_on_migr:
            keep_vm_migrate_on_its_host(vmname)
        
        if vmtype == 1:
            hyper_list = az_distribution(vmname)
        
        elif vmtype == 2:
            hyper_list = cn_distribution(vmname)
        
        elif vmtype == 3:
            hyper_list = even_distribution(vmname)
            
        elif vmtype == 4: 
            hyper_list = triple_distribution(vmname)
        #print(vmname,vmtype)
        #print(vmname, set([dUseHyper[i]['az'] for i in hyper_list]))
        main_distributuion_function(hl=hyper_list,vm=vmname)
        
        
def count_after_distrib(h,vm):
    az = dUseHyper[h]['az']
    vm_group = dUseVms[vm]['group']
    if vm_group != '':
        dUseGroups[vm_group]['az'][az] = dUseGroups[vm_group]['az'][az] + 1
    vm_cpu = dUseVms[vm]['cpu']
    vm_ram = dUseVms[vm]['ram']
    dUseHyper[h]['free_ram'] = dUseHyper[h]['free_ram'] - vm_ram
    dUseHyper[h]['free_cpu'] = dUseHyper[h]['free_cpu'] - vm_cpu
    dUseHyper[h]['vms'].append(vm)
    dUseHyper[h]['vm_count']=len(dUseHyper[h]['vms'])
    dUseVms[vm]['new_cn']=h
    pass


def main_distributuion_function(hl, vm):
    top_list=[]
    for h in hl:
        if dUseHyper[h]['migr']:
            print('ouch! migration')
            continue
        pram = dUseHyper[h]['free_ram']/dUseHyper[h]['total_ram']
        pcpu = dUseHyper[h]['free_cpu']/dUseHyper[h]['total_cpu']
        res_score = (0.00001 * pcpu + 1 * pram)
        top_list.append([res_score,h])
    best_buddy = sorted(top_list, reverse=True)[:1][0][1]
    print(top_list)
    count_after_distrib(h=best_buddy,vm=vm)
    
    


def keep_vm_migrate_on_its_host(vm):
    #print(vm, 'keep')
    pass


def az_distribution(vm):
    #print(vm, 'AZ')
    pass

def cn_distribution(vm):
    #print(vm, 'CN')
    pass



def even_distribution(vm):
    
    group = dUseVms[vm]['group']
    #ga1 = dUseGroups[group]['az1']
    #ga2 = dUseGroups[group]['az2']
    #ga3 = dUseGroups[group]['az3']
    if dUseGroups[group]['az_even'] == '':
        dUseGroups[group]['az_even'] = get_best_availability_zone(group=group)
    if dUseGroups[group]['az_odd'] == '':
        dUseGroups[group]['az_odd'] = get_best_availability_zone(group=group)
    odd_zone = dUseGroups[group]['az_odd']
    even_zone = dUseGroups[group]['az_even']
    stripped_name = vm.strip(group+'-')
    print(group, )
    regexp = re.compile(r"\d\d-\d\d")
    if regexp.match(stripped_name):
        vmnumber_in_group = int(stripped_name.split('-')[1])
    else:
        vmnumber_in_group = int(stripped_name)
    #print(vm, vmnumber_in_group)
    if vmnumber_in_group % 2 == 0:
        az_to_land = even_zone
    else:
        az_to_land = odd_zone
    rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] == az_to_land]
    return rl





def triple_distribution(vm):
    #print(vm, 'TRIPLE')
    group = dUseVms[vm]['group']
    #ga1 = dUseGroups[group]['az1']
    #ga2 = dUseGroups[group]['az2']
    #ga3 = dUseGroups[group]['az3']
    vmnumber_in_group = int(vm.strip(group+'-'))
    az_to_land = (vmnumber_in_group % 3) + 1
    rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] == az_to_land]
    return rl



def get_best_availability_zone(group):
    azl = [i for i in [1,2,3] if i not in [dUseGroups[group]['az_odd'], dUseGroups[group]['az_even']]]
    alist=[]
    for a in azl:
        memory=sum([dUseHyper[i]['free_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == a and\
                    not dUseHyper[i]['migr']])
        alist.append([memory,a])
    #print(sorted(alist, reverse=True))
    az = sorted(alist, reverse=True)[:1][0][1]
    print(az)
    return az

In [262]:
#dUseHyper

In [263]:
#dRealHyper['cn-d720.egron.tnx']['memory_size']

In [264]:
###
### создаем виртуальную среду на основе базовых словарей
###

dVirtualHyper = create_dict_computes_clean(z1=lAZ1,z2=lAZ2,z3=lAZ3,migr=lMigr,cn=lCN)
dVirtualVm = create_dict_vm_with_groups(vdict=dRealVm,gdict=dGroups)
dVirtualGroups = create_dict_groups_via_az(gdict=dGroups)
ml = [i for i in dVirtualHyper.keys() if dVirtualHyper[i]['migr'] == True]
vms_on_migr = [i for j in ml for i in dVirtualVm.keys() if dVirtualVm[i]['cn'] == j]

In [265]:
#dVirtualHyper

In [266]:
#for g in dGroups.keys():
#    virtvm = len([i for i in dVirtualVm.keys() if dVirtualVm[i]['group'] == g])
#    print(g,virtvm,dGroups[g]['vm_count'])

In [267]:
#abr = [i for i in dVirtualVm.keys() if dVirtualVm[i]['group'] != '']
#len(abr)

In [268]:
#lGroupsNull = [i for i in dVirtualGroups.keys() if dVirtualGroups[i]['vms'] == []]
#lGroupsNull

In [269]:
#dVirtualHyper

In [270]:
#rl=[]
#for v in dVirtualVm.keys():
#    vmname = v
#    if dVirtualVm[v]['group'] != '':
#        vmtype = dVirtualGroups[dVirtualVm[v]['group']]['type']
#    else:
#        vmtype= int('0')
#    rl.append([vmname,vmtype])

In [271]:
#sorted(rl, key = lambda x: float(x[1]), reverse=True)

In [272]:
#ml = [i for i in dVirtualHyper.keys() if dVirtualHyper[i]['migr'] == True]

In [273]:
#vms_on_migr = [i for j in ml for i in dVirtualVm.keys() if dVirtualVm[i]['cn'] == j]

In [274]:
#[[i,dRealHyper[i]['running_vms']] for i in dVirtualHyper.keys() if dVirtualHyper[i]['migr'] == True]

In [275]:
#sorted(vms_on_migr)

In [276]:
distributuion_init(gdict=dVirtualGroups,vdict=dVirtualVm,hdict=dVirtualHyper,vms_on_migr=vms_on_migr)

ouch! migration
ouch! migration
ouch! migration
[[1.00001, 'cn-daa0.egron.tnx'], [1.00001, 'cn-db28.egron.tnx'], [1.00001, 'cn-db32.egron.tnx'], [1.00001, 'cn-db38.egron.tnx'], [1.00001, 'cn-db3e.egron.tnx'], [1.00001, 'cn-dc7c.egron.tnx'], [1.00001, 'cn-dc84.egron.tnx'], [1.00001, 'cn-dc8e.egron.tnx'], [1.00001, 'cn-dca8.egron.tnx'], [1.00001, 'cn-dce2.egron.tnx'], [1.00001, 'cn-e13a.egron.tnx'], [1.00001, 'cn-e4b8.egron.tnx'], [1.00001, 'cn-e4c2.egron.tnx']]
ouch! migration
ouch! migration
ouch! migration
ouch! migration
ouch! migration
[[1.00001, 'cn-d720.egron.tnx'], [1.00001, 'cn-d724.egron.tnx'], [1.00001, 'cn-da14.egron.tnx'], [1.00001, 'cn-da16.egron.tnx'], [1.00001, 'cn-da1a.egron.tnx'], [1.00001, 'cn-da1e.egron.tnx'], [1.00001, 'cn-da32.egron.tnx'], [1.00001, 'cn-da44.egron.tnx'], [1.00001, 'cn-da46.egron.tnx'], [1.00001, 'cn-da74.egron.tnx'], [1.00001, 'cn-db0a.egron.tnx'], [1.00001, 'cn-db74.egron.tnx'], [1.00001, 'cn-db76.egron.tnx'], [1.00001, 'cn-dc6c.egron.tnx'], [1.000

TypeError: 'NoneType' object is not iterable

In [251]:
[[dUseGroups[i]['az_even'],dUseGroups[i]['az_odd']] for i in dUseGroups.keys() if dUseGroups[i]['type'] == 3]

[[3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2],
 [3, 2]]

In [49]:
#azl = [[i, i+1] for i in [1,2,3] if i not in ['','']]

In [50]:
#sorted(azl,reverse=True)[:1][0][0]

In [51]:
#dUseHyper

In [257]:
#dUseHyper
[dUseGroups[i] for i in dUseGroups.keys() if dUseGroups[i]['type'] == 3]

[{'az': {1: 0, 2: 3, 3: 3},
  'az_even': 3,
  'az_odd': 2,
  'type': 3,
  'vm_count': 6,
  'vms': ['isurz-mq-06',
   'isurz-mq-05',
   'isurz-mq-04',
   'isurz-mq-03',
   'isurz-mq-02',
   'isurz-mq-01']},
 {'az': {1: 0, 2: 5, 3: 5},
  'az_even': 3,
  'az_odd': 2,
  'type': 3,
  'vm_count': 10,
  'vms': ['ir-bd-01-01',
   'ir-bd-01-02',
   'ir-bd-01',
   'ir-bd-08',
   'ir-bd-07',
   'ir-bd-06',
   'ir-bd-05',
   'ir-bd-04',
   'ir-bd-03',
   'ir-bd-02']},
 {'az': {1: 0, 2: 3, 3: 3},
  'az_even': 3,
  'az_odd': 2,
  'type': 3,
  'vm_count': 6,
  'vms': ['ppoz-process-core-06',
   'ppoz-process-core-05',
   'ppoz-process-core-04',
   'ppoz-process-core-03',
   'ppoz-process-core-02',
   'ppoz-process-core-01']},
 {'az': {1: 0, 2: 3, 3: 3},
  'az_even': 3,
  'az_odd': 2,
  'type': 3,
  'vm_count': 6,
  'vms': ['ppoz-gmp-process-06',
   'ppoz-gmp-process-05',
   'ppoz-gmp-process-04',
   'ppoz-gmp-process-03',
   'ppoz-gmp-process-02',
   'ppoz-gmp-process-01']},
 {'az': {1: 0, 2: 3, 3: 3

In [258]:
    azl=[1,2,3]
    alist=[]
    for a in azl:
        memory=sum([dUseHyper[i]['free_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == a and\
                    not dUseHyper[i]['migr']])
        alist.append([memory,a])
    print(sorted(alist, reverse=True))
    az = sorted(alist, reverse=True)[:1][0][1]
    print(az)

[[17277238, 3], [5592162, 2], [5483877, 1]]
3
